In [ ]:
import copy
import math
import itertools
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def NoMove_stationary(lambd, mu, sigma, K, lambd_p):
  b_stat = mu/lambd
  if b_stat<K:
    a_stat = (K - b_stat)/(1 + (lambd_p*K/sigma))
    return [a_stat, b_stat]
  elif b_stat>K:
    return [0, K]

In [ ]:
def HomogeneousSLV_Gillespie(x0, N, t_end, d1, bb, d2, p2, p1):
    t = [0]                                # time array
    a, b, e = [x0[0]], [x0[1]], [x0[2]]    # predators, preys, empty space

    while(t[-1] < t_end):
        a_t = a[-1]
        b_t = b[-1]
        e_t = e[-1]

        # reaction 1:  a  -> e
        # reaction 2: a+b -> a+a
        # reaction 3: b+e -> b+b
        # rection  4:  b  -> e
        # reaction 5: a+b -> a+e
        rates = np.array([d1*a_t, 2*p1*a_t*b_t/N, 2*bb*b_t*e_t/N, d2*b_t, 2*p2*a_t*b_t/N,]) # rates of the reactions
        norm_rates = rates/np.sum(rates)
        #print(norm_rates)

        tau = np.random.exponential(scale = 1/np.sum(rates))    # exponentially distributed, with mean value 1/sum(rates)
        t.append(t[-1] + tau)

        index = np.random.choice(np.arange(5), p = norm_rates)  # choice of the type of reaction
        if index == 0:      # reaction 1
            a.append(a_t-1)
            b.append(b_t)
            e.append(e_t+1)
        elif index == 1:    # reaction 2
            a.append(a_t+1)
            b.append(b_t-1)
            e.append(e_t)
        elif index == 2:    # reaction 3
            a.append(a_t)
            b.append(b_t+1)
            e.append(e_t-1)
        elif index == 3:    # reaction 4
            a.append(a_t)
            b.append(b_t-1)
            e.append(e_t+1)
        elif index == 4:    # reaction 5
            a.append(a_t)
            b.append(b_t-1)
            e.append(e_t+1)
        else:
            print("Error")
            break

    return [t, a, b, e]

In [ ]:
def plot_Gillespie(res, t_end, z_stat):
    t, a, b = res[0], res[1], res[2]


    fig = make_subplots(rows=2, cols=2,  specs=[[{}, {}], [{"colspan":2}, None]], subplot_titles=('Predators Dynamics', 'Preys Dynamics', 'Predators+Preys Dynamics'))

    fig.add_trace(go.Scatter(x=t, y=a, name='predators', mode='lines', marker=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0,t_end], y=2*[z_stat[0]], mode='lines', line_dash='dash', name='Predators steady state', marker=dict(color='red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=t, y=b, name='preys', mode='lines', marker=dict(color='dodgerblue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=[0,t_end], y=2*[z_stat[1]], mode='lines', line_dash='dash', name='Preys steady state', marker=dict(color='orange')), row=1, col=2)

    fig.add_trace(go.Scatter(x=t, y=a, name='predators', mode='lines', marker=dict(color='blue'), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=[0,t_end], y=2*[z_stat[0]], mode='lines', line_dash='dash', name='Predators steady state', marker = dict(color='red'), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=b, name='preys', mode='lines', marker=dict(color='dodgerblue'), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=[0,t_end], y=2*[z_stat[1]], mode='lines', line_dash='dash', name='Preys steady state', marker=dict(color='orange'), showlegend=False), row=2, col=1)

    fig.update_layout(title_text='Homogeneous LV model - Gillespie algorithm', hovermode='x unified', height=1000)
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Abundance')
    fig.show()

## Stochastic simulation - Gillespie

Here we use the Gillespie algorithm to simulate a stochastic version of the system, which displays the same stationary points.

<br>

Possible reactions:
$$
A \xrightarrow{\mathit{d_1}} E \\
B+E \xrightarrow{\mathit{b}} B+B \\
B \xrightarrow{\mathit{d_2}} E \\
A+B \xrightarrow{\mathit{p_2}} A+E \\
A+B \xrightarrow{\mathit{p_1}} A+A
$$

<br><br>

Master equation, continuous limit, Kramer-Moyal expansion $\Rightarrow$ Fokker Plank and Langevin equation.

Moreover if $N \rightarrow \infty$ and we neglect the noise we get the mean field approximation, those equations are nothing but the **Lotka-Volterra equations**.

$$
\begin{cases}
  \dot{a}(t) = 2\,p_1\,a(t)\,b(t)-d_1\,a(t) \\
  \dot{b}(t) = r\,b(t)\,(1-b(t)/\tilde{K})-2(p_1+p_2+b)\,a(t)\,b(t)
\end{cases}
$$

<br>

where $r=(2b-d_2)$ and $\tilde{K}=\frac{2b-d_2}{2b}$.

<br><br>


Instead, in our case:

$$
\begin{cases}
  \dot{a}(t) = \lambda\,a(t)\,b(t)-\mu\,a(t) \\
  \dot{b}(t) = b\,b(t)\,(1-b(t)/K)-(\sigma/K+\lambda^{'})\,a(t)\,b(t)
\end{cases}
$$


So we have to set:

$$
p_1 = \lambda/2 \\
\mu=d_1 \\
p_2=\frac{\lambda^{'}- \lambda}{2} \\
d_2= \frac{\sigma}{K} (1-K) \\
b=\frac{\sigma}{2K}
$$


# Run!

#### If $\mu/\lambda > K \Rightarrow $$\;(0, K)$  stationary point

In [ ]:
N = 1000          # Total population

In [ ]:
# mu/lambd = 1.0 > K = 0.8

mu = 0.2          # predator mortality
lambd = 0.2         # rate of eating
sigma = 0.5       # reproduction rate of preys
K = 0.8           # carrying capacity
lambd_p = 0.6       # predation rate

d1 = mu
bb = sigma/(2*K)
d2 = (sigma/K)*(1-K)
p2 = 0.5*(lambd_p-lambd)
p1 = 0.5*lambd

d1, bb, d2, p2, p1

(0.2, 0.3125, 0.12499999999999997, 0.19999999999999998, 0.1)

In [ ]:
z_stat = NoMove_stationary(lambd, mu, sigma, K, lambd_p)
z_stat = N*np.array(z_stat)
z_stat

array([  0., 800.])

In [ ]:
t_end = 1000
x0 = [0.1*N, 0.4*N, 0.5*N]    # initial condition (#predators, #prey, #empty sites)

results = HomogeneousSLV_Gillespie(x0, N, t_end, d1, bb, d2, p2, p1)
plot_Gillespie(results, t_end, z_stat)

Output hidden; open in https://colab.research.google.com to view.

#### If $\mu/\lambda < K \Rightarrow $ $\; \left(\frac{K- \mu/\lambda }{1+\lambda^{'}K/\sigma}, \frac{\mu}{\lambda} \right)$  stationary point

In [ ]:
N = 1000          # Total population

In [ ]:
# mu/lambd = 0.4 < K = 0.8

mu = 0.2          # predator mortality
lambd = 0.5         # rate of eating
sigma = 0.5       # reproduction rate of preys
K = 0.8           # carrying capacity
lambd_p = 0.6       # predation rate

d1 = mu
bb = sigma/(2*K)
d2 = (sigma/K)*(1-K)
p2 = 0.5*(lambd_p-lambd)
p1 = 0.5*lambd

d1, bb, d2, p2, p1

(0.2, 0.3125, 0.12499999999999997, 0.04999999999999999, 0.25)

In [ ]:
z_stat = NoMove_stationary(lambd, mu, sigma, K, lambd_p)
z_stat = N*np.array(z_stat)
z_stat

array([204.08163265, 400.        ])

In [ ]:
t_end = 1000
x0 = [0.1*N, 0.1*N, 0.8*N]    # initial condition (#predators, #prey, #empty sites)

results = HomogeneousSLV_Gillespie(x0, N, t_end, d1, bb, d2, p2, p1)
plot_Gillespie(results, t_end, z_stat)

Output hidden; open in https://colab.research.google.com to view.

#### Animated plot


In [2]:
N = 1000          # Total population

In [4]:
sigma / lambd_p

0.8333333333333334

In [3]:
# mu/lambd = 0.4

mu = 0.2          # predator mortality
lambd = 0.5         # rate of eating
sigma = 0.5       # reproduction rate of preys
lambd_p = 0.6       # predation rate

d1 = mu
p2 = 0.5*(lambd_p-lambd)
p1 = 0.5*lambd


t_end = 80 # 100
x0 = [0.1*N, 0.1*N, 0.8*N]    # initial condition (#predators, #prey, #empty sites)

In [ ]:
# Version for images - Presentation


fig = make_subplots(rows = 1, cols = 2, subplot_titles = ('Predators Dynamics', 'Preys Dynamics'))

Ks = np.linspace(10e-2, 1., 15)
for K in Ks:
    bb = sigma/(2*K)
    d2 = (sigma/K)*(1-K)
    #print(d1, bb, d2, p2, p1)


    results = HomogeneousSLV_Gillespie(x0, N, t_end, d1, bb, d2, p2, p1)
    t, a, b = results[0], results[1], results[2]

    z_stat = NoMove_stationary(lambd, mu, sigma, K, lambd_p)
    z_stat = N*np.array(z_stat)

    trace = go.Scatter(x = t,
                       y = a,
                       name = 'a',
                       mode = 'lines',
                       hovertemplate = '%{y}<extra></extra>',
                       visible = False)
    fig.add_trace(trace, row = 1, col = 1)

    trace = go.Scatter(x = t,
                       y = b,
                       name = 'b',
                       mode = 'lines',
                       hovertemplate = '%{y}<extra></extra>',
                       visible = False)
    fig.add_trace(trace, row = 1, col = 2)

    trace = go.Scatter(x = [0,t_end], y = 2*[z_stat[0]], name = 'a*', mode='lines', line_dash='dash', marker=dict(color='blue'), hovertemplate = '%{y}<extra></extra>', visible = False)
    fig.add_trace(trace, row=1, col=1)
    trace = go.Scatter(x = [0,t_end], y = 2*[z_stat[1]], name = 'b*', mode='lines', line_dash='dash', marker=dict(color='orange'), hovertemplate = '%{y}<extra></extra>', visible = False)
    fig.add_trace(trace, row = 1, col = 2)


fig.data[0].visible, fig.data[1].visible, fig.data[2].visible, fig.data[3].visible = True, True, True, True

steps = []
for i in range(0, len(fig.data), 4):
    step = dict(method = 'update',
                args = [{'visible': [False]*len(fig.data)}],
                label = str(round(Ks[int(i/4)], 2)))
    step['args'][0]['visible'][i], step['args'][0]['visible'][i+1], step['args'][0]['visible'][i+2], step['args'][0]['visible'][i+3] = True, True, True, True
    steps.append(step)


fig.update_layout(legend = dict(y = 1.2),
                  sliders = [dict(steps = steps, currentvalue={"prefix": "K="}, pad = dict(t = 75, b = 4, l = 40))])
fig.update_xaxes(title_text = 'Time')  # , row = 1, col = 1
fig.update_yaxes(title_text = 'Abundance', range = [-0.005,350], row = 1, col = 1)
fig.update_yaxes(title_text = 'Abundance', range = [-0.005,750], row = 1, col = 2)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

#### Gillespie simultation - No function

In [ ]:
# N = 1000        # Total population

# ### using lists since we don't know the size of the final arrays
# t = [0]        # time array
# a = [0.1*N]    # predators
# b = [0.4*N]    # preys
# e = [0.5*N]    # empty space

# t_end = 1000
# while(t[-1] < t_end):
#     a_t = a[-1]
#     b_t = b[-1]
#     e_t = e[-1]

#     # reaction 1:  a  -> e
#     # reaction 2: a+b -> a+a
#     # reaction 3: b+e -> b+b
#     # rection  4:  b  -> e
#     # reaction 5: a+b -> a+e
#     rates = np.array([d1*a_t, 2*p1*a_t*b_t/N, 2*bb*b_t*e_t/N, d2*b_t, 2*p2*a_t*b_t/N,]) # rates of the reactions
#     norm_rates = rates/np.sum(rates)

#     tau = np.random.exponential(scale = 1/np.sum(rates))    # exponentially distributed, with mean value 1/sum(rates)
#     t.append(t[-1] + tau)

#     index = np.random.choice(np.arange(5), p = norm_rates)  # choice of the type of reaction
#     if index == 0:      # reaction 1
#         a.append(a_t-1)
#         b.append(b_t)
#         e.append(e_t+1)
#     elif index == 1:    # reaction 2
#         a.append(a_t+1)
#         b.append(b_t-1)
#         e.append(e_t)
#     elif index == 2:    # reaction 3
#         a.append(a_t)
#         b.append(b_t+1)
#         e.append(e_t-1)
#     elif index == 3:    # reaction 4
#         a.append(a_t)
#         b.append(b_t-1)
#         e.append(e_t+1)
#     elif index == 4:    # reaction 5
#         a.append(a_t)
#         b.append(b_t-1)
#         e.append(e_t+1)
#     else:
#         print("Error")
#         break